# flexible environment curve generation

In [ ]:
import astropy as ap
from astropy import coordinates
from astropy.io import fits
import bisect
import matplotlib.pyplot as plt
import multiprocessing as mp
import numpy as np
import os
import pandas as pd
import pickle as pkl

## preprocess GAMA-like SLICS mock positions and redshifts

picking one LOS for now, can rerun for others or convert to script

In [ ]:
LOS = 42
loskey = str(LOS)
field = loskey
key = loskey

toppath = '/media/CRP6/Cosmology/recidivator/SLICS/'

filter for data quality -- only needs to be done once, but it's fast

In [ ]:
# frompath = '/media/CRP6/Cosmology/data/SLICS_GAMA/'

# with fits.open(frompath+'FullGalCatalogGAMA_Lcorr_clean_LOS'+loskey+'.fits') as data:
# #     print(data[1].header)
#     df = pd.DataFrame(data[1].data)
    
# # plt.scatter(df['x_arcmin_clustering'], df['y_arcmin_clustering'], s=1, alpha=0.1)
# # plt.xlim(0, 600)
# # plt.ylim(0, 600)

# high_q = pd.DataFrame()
# high_q['RA'] = df['x_arcmin_clustering'] / 60.
# high_q['DEC'] = df['y_arcmin_clustering'] / 60.
# high_q['Z'] = df['z_spectroscopic']
# high_q.index.names = ['CATAID']

# # plt.scatter(high_q['RA'], high_q['DEC'], s=1, alpha=0.1)
# # plt.xlim(0, 10)
# # plt.ylim(0, 10)

# gf = {}
# gf[key] = high_q
# # # gf['G02'] = high_q.loc[(high_q['RA'] > 30.2) & (high_q['RA'] < 38.8) & (high_q['DEC'] > -10.5) & (high_q['DEC'] < -3.72)]
# # # gf['G09'] = high_q.loc[(high_q['RA'] > 129.0) & (high_q['RA'] < 141.0) & (high_q['DEC'] > -2.) & (high_q['DEC'] < 3.)]
# # # gf[key] = high_q.loc[(high_q['RA'] > minx) & (high_q['RA'] < maxx) & (high_q['DEC'] > miny) & (high_q['DEC'] < maxy)]
# # # gf['G15'] = high_q.loc[(high_q['RA'] > 211.5) & (high_q['RA'] < 223.5) & (high_q['DEC'] > -2.) & (high_q['DEC'] < 3.)]

# gf[key].to_csv(toppath+'flexible_envirocurves/LOS'+loskey+'inputs.csv')

# # # plt.scatter(gf[key]['RA'], gf[key]['DEC'], s=1)
# # # plt.xlim(minx, maxx)
# # # plt.ylim(miny, maxy)

## chunk up data

In [ ]:
maxang = ap.units.Quantity(2.5, 'deg')

#max number of galaxies per chunked file
maxfile = 2**14
share_index = range(maxfile)

first, chunk up big LOS file, which is fast

In [ ]:
# old way to get neighbors on real data

# def get_neighbors(keyno):
#     field = list(gf.keys())[keyno]
#     small_piece = pd.read_csv(toppath+'flexible_envirocurves/LOS'+field+'inputs.csv', index_col=['CATAID'])
# #     print(small_piece.iloc[99])
#     coords = ap.coordinates.SkyCoord(small_piece['RA'], small_piece['DEC'], unit='deg')
#     #warning: slow!!! (not actually that slow)
#     output = ap.coordinates.search_around_sky(coords, coords, maxang)
#     pkl.dump(output, open(toppath+'flexible_envirocurves/neighbors_'+field+'_allz.pkl', 'wb'))
#     return(output)

In [ ]:
def break_down_neighbor_input(keyno):
    big_in = pd.read_csv(toppath+'flexible_envirocurves/LOS'+field+'inputs.csv', index_col=['CATAID'])
    big_out = big_in.copy()
    nbig = len(big_in)# total number of galaxies in big file
    nfile = 0
    paircount = []
    while nfile * maxfile <= nbig:
        glob_ind_min = nfile * maxfile# min real index for this chunk
        glob_ind_max = (nfile + 1) * maxfile
        if nbig < glob_ind_max:
            chunk_ind_max = nbig - glob_ind_min
            glob_ind_max = nbig
        else:
            chunk_ind_max = maxfile
        paircount.append((glob_ind_min, glob_ind_max))
        small_piece = big_in[glob_ind_min : glob_ind_max]
        big_out['chunk_'+str(nfile)] = np.nan
        big_out['chunk_'+str(nfile)][glob_ind_min : glob_ind_max] = share_index[:chunk_ind_max]
        small_piece.to_csv(toppath+'flexible_envirocurves/LOS'+field+'chunk'+str(nfile)+'.csv')
        print('separated out chunk '+str(nfile)+' of LOS '+field+': '+str(paircount[-1]))
        nfile += 1
#     paircount = nfile
    big_out.to_csv(toppath+'flexible_envirocurves/LOS'+field+'allchunks.csv')
    minx, maxx = min(big_in['RA']), max(big_in['RA'])
    miny, maxy = min(big_in['DEC']), max(big_in['DEC'])
    extrema = (minx, maxx, miny, maxy)
    return nfile, paircount, extrema

In [ ]:
# nps = 1#len(gf.keys()) #mp.cpu_count()
# pool = mp.Pool(nps)
# paircounts = pool.map(break_down_neighbor_input, range(1))#range(len(gf.keys())))
# # print(paircounts)
# with open(toppath+'flexible_envirocurves/LOS'+field+'chunkinfo.csv', 'wb') as chunkinfo:
#     pkl.dump(paircounts, chunkinfo)

set bounds for normalizing neighbors per area and magic numbers for manipulating chunked files

In [ ]:
with open(toppath+'flexible_envirocurves/LOS'+field+'chunkinfo.csv', 'rb') as chunkinfo:
    paircounts = pkl.load(chunkinfo)

nfile = paircounts[0][0]
indends = paircounts[0][1]
(minx, maxx, miny, maxy) = paircounts[0][2]

## find neighbors with astropy

must be parallelized because still pretty slow, but should not need to run more than once

In [ ]:
chunk_pairs = {}
pairkey = 0
for i in range(nfile):
    for j in range(nfile):
        chunk_pairs[pairkey] = (i, j)
        pairkey += 1

In [ ]:
def get_neighbors_breakup(keyno):
    field = loskey
    (a, b) = chunk_pairs[keyno]
    small_a = pd.read_csv(toppath+'flexible_envirocurves/LOS'+field+'chunk'+str(a)+'.csv', index_col=['CATAID'])
    small_b = pd.read_csv(toppath+'flexible_envirocurves/LOS'+field+'chunk'+str(b)+'.csv', index_col=['CATAID'])
#     big_piece = pd.read_csv(toppath+'flexible_envirocurves/LOS'+field+'inputs.csv', index_col=['CATAID'])
#     small_piece = break_down_neighbor_input(big_piece)
#     print(small_piece.iloc[99])
    coords_a = ap.coordinates.SkyCoord(small_a['RA'], small_a['DEC'], unit='deg')
    coords_b = ap.coordinates.SkyCoord(small_b['RA'], small_b['DEC'], unit='deg')
    #warning: slow!!! (not actually that slow)
    print('started finding neighbors')
    output = ap.coordinates.search_around_sky(coords_a, coords_b, maxang)
    print('finished finding neighbors')
    pkl.dump(output, open(toppath+'flexible_envirocurves/neighbors_LOS'+field+'chunk'+str(a)+'Xchunk'+str(b)+'allz.pkl', 'wb'))
    return

In [ ]:
# nps = 35#pairkey #mp.cpu_count()
# pool = mp.Pool(nps)
# pool.map(get_neighbors_breakup, range(pairkey))

## parse the neighbor info to make curves

have to do this for every set of distances because too memory intensive

In [ ]:
# try with many more choices for this!
ndist = 100# 128
# distance_evaluation_points = np.exp(np.linspace(np.log(2.5 / 60. / 60.), np.log(2.5), ndist+2)[1:-1])
distance_evaluation_points = np.linspace(0., maxang.value, ndist+2)[1:-1]
# pos = bisect.bisect(distance_evaluation_points, maxang.value)
# distance_evaluation_points = distance_evaluation_points[:pos]
# n_dists = len(distance_evaluation_points)
# iter_over_dists = enumerate(distance_evaluation_points)
# ndist = range(len(distance_evaluation_points))
# with open(toppath+'flexible_envirocurves/eval'+str(ndist)+'dists_LOS'+field+'.pkl', 'wb') as savedists:
#     pkl.dump(distance_evaluation_points, savedists)
print(distance_evaluation_points)

In [ ]:
def segment(r, d, theta=None):
    if theta == None:
        theta = 2. * np.arccos(d / r)
    return r**2 * (theta - np.sin(theta)) / 2.

def sector(r, d, theta=None):
    if theta == None:
        theta = np.arcsin(d / r)
    return r**2 * theta / 2.

# this throws an error at the points used to define minx, maxx, miny, maxy
def area(r, x, y, minx, maxx, miny, maxy, vb=True):
    lx = x - minx
    ux = maxx - x
    ly = y - miny
    uy = maxy - y
    distances = np.array([lx, ux, ly, uy])
    #print(distances)
    condition = (distances >= r)
    ntrue = sum(condition)
    if ntrue == 4:
        return np.pi * r**2
    elif ntrue == 3:
        return np.pi * r**2 - segment(r, min(distances))
    elif ntrue == 2:
        if vb: print('radii should be chosen so that these cannot be parallel, \
                but will at some point add in a check for this')
        distx = min(distances[:2])
        disty = min(distances[-2:])
        if np.sqrt(distx**2 + disty**2) < r:
            thetax = np.arcsin(distx / r)
            thetay = np.arcsin(disty / r)
            areax = distx * r * np.cos(thetax) / 2.
            areay = disty * r * np.cos(thetay) / 2.
            return sector(r, distx, theta=thetax) + sector(r, disty, theta=thetay) + \
                            sector(r, r, theta=np.pi / 2.) + distx * disty + areax + areay
        else:
            return np.pi * r**2 - segment(r, distx) - segment(r, disty)
    else:
        if vb: print('this case should not happen because we did not consider radii \
                beyond half the shortest side of the footprint,\
                but will at some point deal with this case')
        return None


# ## Calculates volume normalized environment
# def calc_env(ind):
#     """
#         Runs galenv to calculate galaxy environment.
#         This is set up to run in the multiprocessing so a lot of inputs are
#         not set when you call the function, but are supposed to be defined
#         when running this code.

#         Output: nearest neighbors at a given angular separation.
#     """
#     if opts.run_environment:
#         # Generates environments for GAMA RA/Dec data
#         res = [subsamples[f][s]['CATAID'].values[ind]]
#     if opts.run_particle_environment:
#         # Generates environments for particle RA/Dec data
#         res = [ind]

#     friends = data
#     for dist in try_distances:
#         friends = galenv.nn_finder(friends, data[ind], dist)
#         #print('r/dist', dist, 'x ', data[ind][0], 'y ', data[ind][1], 'minx', minx, 'maxx', maxx, 'miny', miny, 'maxy', maxy)
#         vol = area(dist, data[ind][0], data[ind][1], minx, maxx, miny, maxy, vb=False)
#         #print('vol', vol)
#         res.append(float(len(friends)) / vol)
#     return res
# ###

In [ ]:
for i in range(nfile):
#     with open(toppath+'flexible_envirocurves/LOS'+loskey+'chunk'+str(i)+'.csv', 'rb') as smallfile:
    small_piece = pd.read_csv(toppath+'flexible_envirocurves/LOS'+loskey+'chunk'+str(i)+'.csv', index_col='CATAID')
#     small_piece['neighbor_CATAID'] = small_piece.index
#     small_piece['neighbor_CATAID'] = small_piece.apply(lambda row: [row['neighbor_CATAID']], axis=1)
#     small_piece['neighbor_dist'] = small_piece.apply(lambda row: [0.], axis=1)
#     with open(toppath+'flexible_envirocurves/eval'+str(ndist)+'dists_LOS'+field+'.pkl', 'rb') as savedists:
    which_dists = distance_evaluation_points
#     for dist in which_dists:
#         small_piece[str(dist)] = 0
    for k, dist in enumerate(which_dists):
        small_piece[str(dist)] = np.zeros_like(len(small_piece))
    for j in range(nfile):
        with open(toppath+'flexible_envirocurves/neighbors_LOS'+loskey+'chunk'+str(i)+'Xchunk'+str(j)
                  +'allz.pkl', 'rb') as tomerge:
            all_pairs = pkl.load(tomerge)
        inds_with_neighbors, where_my_neighbors_at = np.unique(all_pairs[0], return_inverse=True)
#         for ind in np.unique(all_pairs[0])[:10]:
        def help_gather_neighbors(ind):
            newrow = np.zeros_like(which_dists)
            if ind in inds_with_neighbors:
#                 ra = small_piece.iloc[ind + indends[i][0]]['RA']
#                 dec = small_piece.iloc[ind + indends[i][0]]['DEC']
#             where_my_neighbors_at = np.where(all_pairs[0] == ind)
#             my_neighbor_inds = all_pairs[1][where_my_neighbors_at == ind]
#             tosave = list(my_neighbor_inds + indends[j][0])
#             print(tosave)
#             small_piece['neighbor_CATAID'].loc[ind + indends[i][0]] += list(my_neighbor_inds + indends[j][0])
#             where_to_save = ind + indends[i][0]
#             CATAIDs_to_save = list(my_neighbor_inds + indends[j][0])
                my_neighbor_dists = all_pairs[2][where_my_neighbors_at == ind].value
#             small_piece['neighbor_dist'].loc[ind + indends[i][0]] += list(my_neighbor_dists)
                sort_dists = sorted(list(my_neighbor_dists))
                for k, dist in enumerate(which_dists):
                    newrow[k] = bisect.bisect(sort_dists, dist)
#                     vol = area(dist, ra, dec, minx, maxx, miny, maxy, vb=False)
#                     newrow[k] = float(pos) / vol
#             out_info = [ind + indends[i][0], CATAIDs_to_save, dists_to_save]
#             return(sorted(dists_to_save))
            return(newrow)
        nps = 35#mp.cpu_count()
        pool = mp.Pool(nps)
        distcounts = pool.map(help_gather_neighbors, range(len(small_piece.index)))
        distcounts = np.array(distcounts).T
        print(str((i, j, np.shape(distcounts))))
        for k, dist in enumerate(which_dists):
            small_piece[str(dist)] += distcounts[k]
#         for dist in which_dists:
#             small_piece[str(dist)] += bisect.bisect(sort_dists, dist)#.loc[ind + indends[i][0]]
#         with open(toppath+'flexible_envirocurves/tmp_LOS'+loskey+'chunk'+str(i)+'Xchunk'+str(j)+'.pkl', 'wb') as tmpfile:
#             pkl.dump(ids_and_dists, tmpfile)
#     small_piece.apply(lambda row: row['neighbor_CATAID'].pop(0), axis=1)
#     small_piece.apply(lambda row: row['neighbor_dist'].pop(0), axis=1)
#     print(small_piece)
    for k, dist in enumerate(which_dists):
        small_piece[str(dist)] = small_piece.apply(lambda row: float(row[str(dist)]) / 
                                                        area(dist, row['RA'], row['DEC'], 
                                                             minx, maxx, miny, maxy, vb=False), axis=1)
    print(small_piece)
#     with open(toppath+'flexible_envirocurves/LOS'+loskey+'chunk'+str(i)+'dists'+str(ndist)+'.csv', 'wb') as smallfile:
    small_piece.to_csv(toppath+'flexible_envirocurves/LOS'+loskey+'chunk'+str(i)+'dists'+str(ndist)+'.csv')

In [ ]:
# # with open(toppath+'flexible_envirocurves/LOS'+loskey+'chunk'+str(0)+'dists'+str(ndist)+'.csv', 'rb') as expfile:
# small_piece = pd.read_csv(toppath+'flexible_envirocurves/LOS'+loskey+'chunk'+str(1)+'dists'+str(ndist)+'.csv', index_col='CATAID')
# print(small_piece)

# # small_piece[str(distance_evaluation_points[0])] += np.ones(len(small_piece.index))

In [ ]:
with open(toppath+'flexible_envirocurves/neighbors_LOS'+loskey+'chunk'+str(1)+'Xchunk'+str(3)+'allz.pkl', 'rb') as tomerge:
    all_pairs = pkl.load(tomerge)
    inds_with_neighbors, where_my_neighbors_at = np.unique(all_pairs[0], return_inverse=True)

In [ ]:
len(where_my_neighbors_at)

In [ ]:
len(inds_with_neighbors)

In [ ]:
len(all_pairs[1][where_my_neighbors_at == 16382])

In [ ]:
test = pd.read_csv(toppath+'flexible_envirocurves/LOS'+loskey+'chunk'+str(0)+'.csv', index_col='CATAID')

In [ ]:
test

In [ ]:
# def gather_neighbors(chunkind):
#     small_piece = pd.read_csv(toppath+'flexible_envirocurves/LOS'+loskey+'chunk'+str(chunkind)+'.csv')
#     small_piece['neighbor_CATAID'] = None
#     small_piece['neighbor_dist'] = None
#     for i in range(nfile):
        
#         where_my_neighbors_at = np.where(all_pairs[0] == ind)
#         my_neighbor_inds = all_pairs[1][where_my_neighbors_at]
#         save_df = df[['CATAID', 'Z']].iloc[my_neighbor_inds]
#     #     my_neighbor_zs = df.iloc[my_neighbor_inds]['Z']
#         neighbor_dists = all_pairs[2][where_my_neighbors_at]
#         save_df['dist'+str(ind)] = neighbor_dists
#     #     nn = len(neighbor_distances)
#     #     res[ind] = sorted(neighbor_distances)
#         output = (ind, df.iloc[ind]['CATAID'], save_df)
#     #     res[ind] = output
#         return(output)

In [ ]:
# for i in range(1):#nfile):
#     all_pairs = pd.read_csv(toppath+'flexible_envirocurves/LOS'+loskey+'chunk'+str(i)+'.csv', index_col=['CATAID'])
#     def help_find_neighbors(ind):
#         where_my_neighbors_at = np.where(all_pairs[0] == ind)
#         my_neighbor_inds = all_pairs[1][where_my_neighbors_at]
#         save_df = df[['CATAID', 'Z']].iloc[my_neighbor_inds]
#     #     my_neighbor_zs = df.iloc[my_neighbor_inds]['Z']
#         neighbor_dists = all_pairs[2][where_my_neighbors_at]
#         save_df['dist'+str(ind)] = neighbor_dists
#     #     nn = len(neighbor_distances)
#     #     res[ind] = sorted(neighbor_distances)
#         output = (ind, df.iloc[ind]['CATAID'], save_df)
#     #     res[ind] = output
#         return(output)
    
#     for j in range(nfile):
#         thing = pkl.load(open(toppath+'flexible_envirocurves/neighbors_LOS'+loskey+'chunk'+str(i)+'Xchunk'+str(j)+'allz.pkl', 'rb'))
        

In [ ]:
# all_pairs = pkl.load(open(toppath+'flexible_envirocurves/neighbors_'+test_key+'_allz.pkl', 'rb'))

In [ ]:
# # add in redshift filtering here or earlier?

# def help_find_neighbors(ind):
#     where_my_neighbors_at = np.where(all_pairs[0] == ind)
#     my_neighbor_inds = all_pairs[1][where_my_neighbors_at]
#     save_df = df[['CATAID', 'Z']].iloc[my_neighbor_inds]
# #     my_neighbor_zs = df.iloc[my_neighbor_inds]['Z']
#     neighbor_dists = all_pairs[2][where_my_neighbors_at]
#     save_df['dist'+str(ind)] = neighbor_dists
# #     nn = len(neighbor_distances)
# #     res[ind] = sorted(neighbor_distances)
#     output = (ind, df.iloc[ind]['CATAID'], save_df)
# #     res[ind] = output
#     return(output)

In [ ]:
# #remember to do for all LOS

# # res = [[]] * len(df.index)
# nps = 20#mp.cpu_count() - 1
# pool = mp.Pool(nps)
# try_neighbor_dists = pool.map(help_find_neighbors, df.index)
# pkl.dump(try_neighbor_dists, open(toppath+'flexible_envirocurves/parsed_neighbor_dists_'+test_key+'.pkl', 'wb'))

## evaluate the curves at angular distances

In [ ]:
the_neighborhood = pkl.load(open(toppath+'flexible_envirocurves/parsed_neighbor_dists_'+test_key+'.pkl', 'rb'))

In [ ]:
the_neighborhood

In [ ]:
def give_redshift_neighbors(ind, delta=0.1):
    my_neighborhood = the_neighborhood[ind][-1]
    all_neighbor_zs = my_neighborhood['Z']
    my_z = all_neighbor_zs[ind]
    close_neighbor_dists = my_neighborhood['dist'+str(ind)][(all_neighbor_zs > my_z-delta) & (all_neighbor_zs < my_z+delta)]
    return(close_neighbor_dists)

In [ ]:
# add in volume normalization here

def help_make_curve(ind):
    curve = [the_neighborhood[ind][1]]#np.empty((ndist))
    (ra, dec) = (df.iloc[ind]['RA'], df.iloc[ind]['DEC'])
#     redshifts = test_data[ind][3]
    distances = sorted(list(give_redshift_neighbors(ind)))
    for dist in distance_evaluation_points:
#         print(i)
        pos = bisect.bisect(distances, dist)
#         print(pos)
#         curve[i] = pos
#         all_curves.iloc[ind][str(i)] = pos
        vol = area(dist, ra, dec, minx, maxx, miny, maxy, vb=False)
        curve.append(float(pos) / vol)
        
    return(curve)

In [ ]:
nps = 10#mp.cpu_count() - 1
pool = mp.Pool(nps)
try_neighbor_curves = pool.map(help_make_curve, df.index)
newdist = np.hstack((np.ones((1)), distance_evaluation_points))
save_curves = np.vstack((newdist, try_neighbor_curves))
pkl.dump(np.array(save_curves), open(toppath+'flexible_envirocurves/zslice_normed_100dist_'+test_key+'.pkl', 'wb'))
pkl.dump(distance_evaluation_points, open(toppath+'flexible_envirocurves/zslice_normed_100dist_'+test_key+'_evaluated_distances.pkl', 'wb'))

# scratch below here

In [ ]:
all_q = pd.read_csv(toppath+'test/one_slice/SpecObjPhot.csv', index_col=['CATAID'])

In [ ]:
high_q = all_q[all_q['NQ'] > 2]

In [ ]:
high_q = high_q.rename(columns={"Unnamed: 0": "SpecObjPhot_index"})

split by field for speed, omit field with nonuniform coverage

sky coordinate limits came from [GAMA website](https://www.astro.ljmu.ac.uk/~ikb/research/gama_fields/) -- must include DEC for this to work!!!

NOTE: this fails on G15 field because it's got too many galaxies

In [ ]:
gf = {}
# gf['G02'] = high_q.loc[(high_q['RA'] > 30.2) & (high_q['RA'] < 38.8) & (high_q['DEC'] > -10.5) & (high_q['DEC'] < -3.72)]
# gf['G09'] = high_q.loc[(high_q['RA'] > 129.0) & (high_q['RA'] < 141.0) & (high_q['DEC'] > -2.) & (high_q['DEC'] < 3.)]
gf['G12'] = high_q.loc[(high_q['RA'] > 174.0) & (high_q['RA'] < 186.0) & (high_q['DEC'] > -3.) & (high_q['DEC'] < 2.)]
# gf['G15'] = high_q.loc[(high_q['RA'] > 211.5) & (high_q['RA'] < 223.5) & (high_q['DEC'] > -2.) & (high_q['DEC'] < 3.)]

## find the neighbors within max angular distance set by GAMA footprint

In [ ]:
n_gal_kept = {}
for key in gf.keys():
    n_gal_kept[key] = len(gf[key])
    gf[key].to_csv(toppath+'flexible_envirocurves/field'+key+'high_q.csv')

In [ ]:
# or key in gf.keys():
    
#     n_gal_kept[key] = len(gf[key])
#     gf[key].to_csv('flexible_envirocurves/field'+key+'high_q.csv')

In [ ]:
easy_inds = {}
for key in gf.keys():
    easy_inds[key] = pd.read_csv(toppath+'flexible_envirocurves/field'+key+'high_q.csv')
    easy_inds[key].index.rename('field_index', inplace=True)

In [ ]:
maxang = ap.units.Quantity(2.5, 'deg')

This is the slow step!!!

In [ ]:
def get_neighbors(keyno):
    field = list(gf.keys())[keyno]
    small_piece = pd.read_csv(toppath+'flexible_envirocurves/field'+field+'high_q.csv')
#     print(small_piece.iloc[99])
    coords = ap.coordinates.SkyCoord(small_piece['RA'], small_piece['DEC'], unit='deg')
    #warning: slow!!! (not actually that slow)
    output = ap.coordinates.search_around_sky(coords, coords, maxang)
    pkl.dump(output, open(toppath+'flexible_envirocurves/neighbors_'+field+'_allz.pkl', 'wb'))
    return(output)

In [ ]:
#still kinda slow
nps = len(gf.keys()) #mp.cpu_count()
pool = mp.Pool(nps)
pool.map(get_neighbors, range(len(gf.keys())))

In [ ]:
# help(ap.coordinates.search_around_sky)

## parse the neighbor info to flexibly make curves

In [ ]:
test_key = 'G12'

In [ ]:
df = pd.read_csv(toppath+'flexible_envirocurves/field'+test_key+'high_q.csv')

In [ ]:
all_pairs = pkl.load(open(toppath+'flexible_envirocurves/neighbors_'+test_key+'_allz.pkl', 'rb'))

In [ ]:
# add in redshift filtering here or earlier?

def help_find_neighbors(ind):
    where_my_neighbors_at = np.where(all_pairs[0] == ind)
    my_neighbor_inds = all_pairs[1][where_my_neighbors_at]
    save_df = df[['CATAID', 'Z']].iloc[my_neighbor_inds]
#     my_neighbor_zs = df.iloc[my_neighbor_inds]['Z']
    neighbor_dists = all_pairs[2][where_my_neighbors_at]
    save_df['dist'+str(ind)] = neighbor_dists
#     nn = len(neighbor_distances)
#     res[ind] = sorted(neighbor_distances)
    output = (ind, df.iloc[ind]['CATAID'], save_df)
#     res[ind] = output
    return(output)

In [ ]:
#remember to do for all fields

# res = [[]] * len(df.index)
nps = 35#mp.cpu_count() - 1
pool = mp.Pool(nps)
try_neighbor_dists = pool.map(help_find_neighbors, df.index)
pkl.dump(try_neighbor_dists, open(toppath+'flexible_envirocurves/parsed_neighbor_dists_'+test_key+'.pkl', 'wb'))

## evaluate the curves at angular distances

In [ ]:
the_neighborhood = pkl.load(open(toppath+'flexible_envirocurves/parsed_neighbor_dists_'+test_key+'.pkl', 'rb'))

In [ ]:
the_neighborhood

In [ ]:
# try with many more choices for this!
ndist = 100
distance_evaluation_points = np.exp(np.linspace(np.log(maxang.value / 60. / 60.), np.log(maxang.value), ndist+2)[1:-1])
# n_dists = len(distance_evaluation_points)
# iter_over_dists = enumerate(distance_evaluation_points)
# ndist = range(len(distance_evaluation_points))

In [ ]:
distance_evaluation_points

In [ ]:
# xpl = np.linspace(0., 2.5, ndist+2)[1:-1]
# plt.scatter(xpl, np.linspace(0., 2.5, ndist+2)[1:-1])
# plt.scatter(xpl, np.log(np.logspace(0., np.log(2.5), ndist+2)[1:-1]))
# plt.scatter(xpl, np.exp(np.linspace(np.log(2.5 / 60. / 60.), np.log(2.5), ndist+2)[1:-1]))
# plt.show()

normalize the curves by neighbors-per-area

In [ ]:
def segment(r, d, theta=None):
    if theta == None:
        theta = 2. * np.arccos(d / r)
    return r**2 * (theta - np.sin(theta)) / 2.

def sector(r, d, theta=None):
    if theta == None:
        theta = np.arcsin(d / r)
    return r**2 * theta / 2.

# this throws an error at the points used to define minx, maxx, miny, maxy
def area(r, x, y, minx, maxx, miny, maxy, vb=True):
    lx = x - minx
    ux = maxx - x
    ly = y - miny
    uy = maxy - y
    distances = np.array([lx, ux, ly, uy])
    #print(distances)
    condition = (distances >= r)
    ntrue = sum(condition)
    if ntrue == 4:
        return np.pi * r**2
    elif ntrue == 3:
        return np.pi * r**2 - segment(r, min(distances))
    elif ntrue == 2:
        if vb: print('radii should be chosen so that these cannot be parallel, \
                but will at some point add in a check for this')
        distx = min(distances[:2])
        disty = min(distances[-2:])
        if np.sqrt(distx**2 + disty**2) < r:
            thetax = np.arcsin(distx / r)
            thetay = np.arcsin(disty / r)
            areax = distx * r * np.cos(thetax) / 2.
            areay = disty * r * np.cos(thetay) / 2.
            return sector(r, distx, theta=thetax) + sector(r, disty, theta=thetay) + \
                            sector(r, r, theta=np.pi / 2.) + distx * disty + areax + areay
        else:
            return np.pi * r**2 - segment(r, distx) - segment(r, disty)
    else:
        if vb: print('this case should not happen because we did not consider radii \
                beyond half the shortest side of the footprint,\
                but will at some point deal with this case')
        return None


# ## Calculates volume normalized environment
# def calc_env(ind):
#     """
#         Runs galenv to calculate galaxy environment.
#         This is set up to run in the multiprocessing so a lot of inputs are
#         not set when you call the function, but are supposed to be defined
#         when running this code.

#         Output: nearest neighbors at a given angular separation.
#     """
#     if opts.run_environment:
#         # Generates environments for GAMA RA/Dec data
#         res = [subsamples[f][s]['CATAID'].values[ind]]
#     if opts.run_particle_environment:
#         # Generates environments for particle RA/Dec data
#         res = [ind]

#     friends = data
#     for dist in try_distances:
#         friends = galenv.nn_finder(friends, data[ind], dist)
#         #print('r/dist', dist, 'x ', data[ind][0], 'y ', data[ind][1], 'minx', minx, 'maxx', maxx, 'miny', miny, 'maxy', maxy)
#         vol = area(dist, data[ind][0], data[ind][1], minx, maxx, miny, maxy, vb=False)
#         #print('vol', vol)
#         res.append(float(len(friends)) / vol)
#     return res
# ###

In [ ]:
# helper to select by redshift:
# easiest to just go +/- 0.1
# then get more sophisticated

def give_redshift_neighbors(ind, delta=0.1):
    my_neighborhood = the_neighborhood[ind][-1]
    all_neighbor_zs = my_neighborhood['Z']
    my_z = all_neighbor_zs[ind]
    close_neighbor_dists = my_neighborhood['dist'+str(ind)][(all_neighbor_zs > my_z-delta) & (all_neighbor_zs < my_z+delta)]
    return(close_neighbor_dists)

In [ ]:
# add in volume normalization here

def help_make_curve(ind):
    curve = [the_neighborhood[ind][1]]#np.empty((ndist))
    (ra, dec) = (df.iloc[ind]['RA'], df.iloc[ind]['DEC'])
#     redshifts = test_data[ind][3]
    distances = sorted(list(give_redshift_neighbors(ind)))
    for dist in distance_evaluation_points:
#         print(i)
        pos = bisect.bisect(distances, dist)
#         print(pos)
#         curve[i] = pos
#         all_curves.iloc[ind][str(i)] = pos
        vol = area(dist, ra, dec, minx, maxx, miny, maxy, vb=False)
        if type(vol) != np.float64:
            print(str((ind, pos))+' at '+str((ra, dec))+' with dist '+str(distances[pos])+' has area '+str(vol)+' of type '+str(type(vol)))
        curve.append(float(pos) / vol)
    return(curve)

In [ ]:
nps = 20#mp.cpu_count() - 1
pool = mp.Pool(nps)
try_neighbor_curves = pool.map(help_make_curve, df.index)
newdist = np.hstack((np.ones((1)), distance_evaluation_points))
save_curves = np.vstack((newdist, try_neighbor_curves))
pkl.dump(np.array(save_curves), open(toppath+'flexible_envirocurves/zslice_normed_log100dist_'+test_key+'.pkl', 'wb'))

In [ ]:
# test_curves = pkl.load(open(toppath+'flexible_envirocurves/zslice_normed_curves_'+test_key+'.pkl', 'rb'))

In [ ]:
# test_curves

# scratch below here

In [ ]:
# do something smarter for getting borders

minx = 174.
maxx = 186.
miny = -3.
maxy = 2.

In [ ]:
# vols = np.empty((ndist))
# for i, dist in enumerate(distance_evaluation_points):
#     vols[i] = area(dist, ra, dec, minx, maxx, miny, maxy, vb=False)

also, subsample redshifts of neighbors to be near redshift of galaxy in question

next steps
- [X] collect neighbors and distances for rudimentary curve
- [X] variable angular distances for evaluation
- [X] normalize for area enclosed within radius
- [X] redshift/depth cutting
- [ ] physical distance conversion

In [ ]:
all_curves = pd.DataFrame(columns = distance_evaluation_points).fillna(0)

In [ ]:
n_gal = len(np.unique(test[0]))
sparse_arr = np.empty((n_gal, n_gal))
sparse_arr[:] = np.nan

In [ ]:
def fill_sparse(ind):
    x, y = test[0][ind], test[1][ind]
    sparse_arr[x][y] = test[2][ind].value
    return sparse_arr

In [ ]:
# for ind in range(len(test[0])):
#     fill_sparse(ind)
n_pair = len(test[0])
nps = 15
pool = mp.Pool(nps)
pool.map(fill_sparse, range(n_pair))

In [ ]:
sparse_arr

In [ ]:
sparse_df = pd.DataFrame(sparse_arr).astype(pd.SparseDtype("float", np.nan))
sparse_df.to_csv('sparse_neighbors_G12.csv')
sparse_df

In [ ]:
sparse_df.isna().sum()

In [ ]:
sparse_test = pd.DataFrame(sparse_arr).astype(pd.SparseDtype("float", np.nan))
sparse_test

In [ ]:
easy_inds['G15']['neighbor_info'] = 
easy_inds['G15']['neighbor_ids'] = []
def fetch_neighbors()

In [ ]:
for key in gf.keys():
    small_piece = gf[field]
    just_ids = small_piece['CATAID']


In [ ]:
def collect_neighbors(field):
    near_dists = pkl.load(open(toppath+'flexible_envirocurves/neighbors_'+field+'_allz.pkl', 'rb'))

## scratch below here

In [ ]:
# np.array([output[0], output[1]]).T

In [ ]:
# all_q.columns

In [ ]:
# print(len(all_q))

In [ ]:
# print(len(high_q))

In [ ]:
# coords1 = ap.coordinates.SkyCoord(small_piece['RA'], small_piece['DEC'], unit='deg')

In [ ]:
# small_piece = df.sample(200)
# coords2 = ap.coordinates.SkyCoord(small_piece['RA'], small_piece['DEC'], unit='deg')

In [ ]:
# plt.scatter(coords1.ra, coords1.dec)
# plt.scatter(coords2.ra, coords2.dec)

In [ ]:
# # warning: slow!!! (not actually that slow)
# output = ap.coordinates.search_around_sky(coords1, coords1, maxang)

In [ ]:
# for o in output:
#     print((len(o), o))

In [ ]:
# plt.hist(output[2].value, bins=100)

In [ ]:
# plt.hist(output[0], bins=len(np.unique(output[0])))